importacion de librerias


In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [2]:
df = pd.read_csv('./data/2024_Accidentalidad.csv', delimiter=';')


In [3]:
df.sample(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
11443,2024S010897,26/03/2024,13:12:00,CALL. VIA LUSITANA / CALL. AGUACATE,150,11,CARABANCHEL,Colisión lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,14.0,Sin asistencia sanitaria,437099.0,4468860.0,N,NaN
3213,2024S002791,25/01/2024,18:15:00,"CMNO. ROBLEDAL, 2",2,9,MONCLOA-ARAVACA,Caída,Despejado,VMU eléctrico,Conductor,De 25 a 29 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,436526.0,4474405.0,N,NaN
7225,2024S006990,24/02/2024,21:17:00,CALL. HERMENEGILDO BIELSA / CALL. CARMEN BRUGUERA,14,12,USERA,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 45 a 49 años,Hombre,14.0,Sin asistencia sanitaria,440449.0,4471192.0,N,NaN
13618,2024S013176,12/04/2024,16:25:00,"AUTOV. M-30, 19XC40",19XC40,9,MONCLOA-ARAVACA,Colisión múltiple,Despejado,Turismo,Conductor,De 21 a 24 años,Hombre,NaN,NaN,438004.0,4475142.0,N,NaN
19072,2024S017001,22/05/2024,9:57:00,"AUTOV. M-30, 32NC02",32NC02,5,CHAMARTÍN,Colisión múltiple,Despejado,Turismo,Conductor,De 45 a 49 años,Mujer,NaN,NaN,443121.0,4481720.0,N,NaN
32946,2024S025990,12/09/2024,20:05:00,AVDA. PESETA / AVDA. CARABANCHEL ALTO,80,11,CARABANCHEL,Colisión lateral,Despejado,Autobus EMT,Conductor,De 45 a 49 años,Mujer,NaN,NaN,435983.0,4468207.0,N,NaN
15271,2024S014363,23/04/2024,20:30:00,"AVDA. DAROCA, 3",3,15,CIUDAD LINEAL,Colisión fronto-lateral,NaN,Turismo,Conductor,De 35 a 39 años,Hombre,NaN,NaN,444124.0,4475823.0,N,NaN
38995,2024S029587,23/10/2024,6:40:00,CALLE RIVAS 39,39,19,VICÁLVARO,Alcance,Despejado,Turismo,Pasajero,De 35 a 39 años,Mujer,1.0,Atención en urgencias sin posterior ingreso,450802.0,4471611.0,N,NaN
20271,2024S018338,29/05/2024,20:45:00,"CALL. SIERRA MENERA, 7",7,13,PUENTE DE VALLECAS,Atropello a persona,Despejado,Turismo,Conductor,De 21 a 24 años,Mujer,14.0,Sin asistencia sanitaria,444012.0,4471531.0,N,NaN
30136,2024S024244,16/08/2024,20:35:00,AVDA. DOCTOR GARCIA TAPIA / CALL. RABAT,1,14,MORATALAZ,Colisión fronto-lateral,Despejado,Turismo,Pasajero,Más de 74 años,Hombre,2.0,Ingreso inferior o igual a 24 horas,446753.0,4473114.0,N,NaN


Analisis del dataset


https://datos.madrid.es/FWProjects/egob/Catalogo/Seguridad/Ficheros/Estructura_ConjuntoDatos_Accidentesv2.pdf


Lo primero que nos damos cuenta cuando leemos el anterior documento es que el año del accidente no viene en la fecha, si no que viene en los 4 primeros char del numero del expediente.


**localizacion** y **numero**: con la localizacion y el numero (numero de la calle) nos parece interesante quedarnosla, pero solo para la parte de entendimiento de los datos del cluster, ya que para el claster es mas util las coordenadas utm


para el codigo del distrito nos va a servir para hacer el cluster, pero como el numero en si no nos dice nada, pues entonces lo que vamos a hacer un mapeo del distrito para entender y tener la correlacion


para el tipo de accidente y para el estado metereologico haremos un mapeo simple de las 13 posibles variables (mapping)


en tipo de vehiculo habia demasiados valroes unicos (33) por lo que hemos intentado generalizar un poco para poder tratar con esta columna, haciendo un mapeo y un posible posterior onehotencoding


para el **rango de edad** vamos


Entendimiento del Dataset


Limpieza


Explicacion de las acciones realizadas


In [49]:
len(df['tipo_vehiculo'].unique())


33

In [4]:
df['tipo_vehiculo'].value_counts()

tipo_vehiculo
Turismo                              27406
Motocicleta hasta 125cc               2668
Furgoneta                             2667
Motocicleta > 125cc                   1469
Autobús                               1264
Camión rígido                          973
Todo terreno                           706
Bicicleta                              670
VMU eléctrico                          665
Ciclomotor                             387
Bicicleta EPAC (pedaleo asistido)      211
Tractocamión                           187
Maquinaria de obras                    151
Otros vehículos con motor              131
Vehículo articulado                     70
Autobús articulado                      69
Sin especificar                         40
Cuadriciclo ligero                      32
Autobus EMT                             17
Ciclo                                   13
Cuadriciclo no ligero                   12
Autocaravana                            11
Otros vehículos sin motor               

In [5]:
map_vehiculos = {
    'Turismo': 'Turismos y vehículos ligeros',
    'Todo terreno': 'Turismos y vehículos ligeros',
    'Autocaravana': 'Turismos y vehículos ligeros',
    'Motocicleta hasta 125cc': 'Motocicletas y ciclomotores',
    'Motocicleta > 125cc': 'Motocicletas y ciclomotores',
    'Ciclomotor': 'Motocicletas y ciclomotores',
    'Bicicleta EPAC (pedaleo asistido)': 'Motocicletas y ciclomotores',
    'Ciclomotor de tres ruedas': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas hasta 125cc': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas > 125cc': 'Motocicletas y ciclomotores',
    'Ciclo de motor L1e-A': 'Motocicletas y ciclomotores',
    'Ciclomotor de dos ruedas L1e-B': 'Motocicletas y ciclomotores',
    'Furgoneta': 'Vehículos de carga',
    'Camión rígido': 'Vehículos de carga',
    'Tractocamión': 'Vehículos de carga',
    'Vehículo articulado': 'Vehículos de carga',
    'Semiremolque': 'Vehículos de carga',
    'Remolque': 'Vehículos de carga',
    'Autobús': 'Vehículos de transporte público',
    'Autobús articulado': 'Vehículos de transporte público',
    'Autobus EMT': 'Vehículos de transporte público',
    'Bicicleta': 'Bicicletas y ciclos no motorizados',
    'Ciclo': 'Bicicletas y ciclos no motorizados',
    'Patinete no eléctrico': 'Bicicletas y ciclos no motorizados',
    'VMU eléctrico': 'Vehículos eléctricos ligeros',
    'Maquinaria de obras': 'Vehículos especializados',
    'Ambulancia SAMUR': 'Vehículos especializados',
    'Otros vehículos con motor': 'Vehículos especializados',
    'Otros vehículos sin motor': 'Vehículos especializados',
    'Sin especificar': 'Sin clasificar/especificar',
    'Cuadriciclo ligero': 'Sin clasificar/especificar',
    'Cuadriciclo no ligero': 'Sin clasificar/especificar'
}

In [6]:
df['tipo_vehiculo'] = df['tipo_vehiculo'].map(map_vehiculos)

Mapeo de rango de edades


In [7]:
map_edad = {
    'Desconocido': -1,
    'Menor de 5 años': 0,
    'De 6 a 9 años': 6,
    'De 10 a 14 años': 10,
    'De 15 a 17 años': 15,
    'De 18 a 20 años': 18,
    'De 21 a 24 años': 21,
    'De 25 a 29 años': 25,
    'De 30 a 34 años': 30,
    'De 35 a 39 años': 35,
    'De 40 a 44 años': 40,
    'De 45 a 49 años': 45,
    'De 50 a 54 años': 50,
    'De 55 a 59 años': 55,
    'De 60 a 64 años': 60,
    'De 65 a 69 años': 65,
    'De 70 a 74 años': 70,
    'Más de 74 años': 74
}

In [8]:
df['rango_edad'] = df['rango_edad'].map(map_edad)

Mapeo de estado meteorológico


In [10]:
df = df.fillna({
    'estado_meteorológico': 'Se desconoce'
})

print(df['estado_meteorológico'].unique())

map_est_met = {
    'Se Desconoce': -1,
    'Despejado': 0,
    'Nublado': 1,
    'Lluvia débil': 2,
    'LLuvia intensa': 3,
    'Granizando': 4,
    'Nevando': 5
}

['Lluvia débil' 'Despejado' 'Se desconoce' 'Nublado' 'LLuvia intensa'
 'Granizando' 'Nevando']


Mapeo Lesividad


In [11]:
print(df['lesividad'].unique())

['Ingreso inferior o igual a 24 horas' 'Sin asistencia sanitaria'
 'Asistencia sanitaria sólo en el lugar del accidente' nan
 'Atención en urgencias sin posterior ingreso'
 'Ingreso superior a 24 horas'
 'Asistencia sanitaria inmediata en centro de salud o mutua'
 'Asistencia sanitaria ambulatoria con posterioridad' 'Fallecido 24 horas']


In [12]:
df = df.fillna({
    'lesividad': 'Sin asistencia sanitaria',
    'cod_lesividad': 0
})

map_lesividad = {
    'Sin asistencia sanitaria': 'NONE',
    'Atención en urgencias sin posterior ingreso': 'LEVE',
    'Ingreso inferior o igual a 24 horas': 'LEVE',
    'Asistencia sanitaria ambulatoria con posterioridad': 'LEVE',
    'Asistencia sanitaria inmediata en centro de salud o mutua': 'LEVE',
    'Asistencia sanitaria sólo en el lugar del accidente': 'LEVE',
    'Ingreso inferior o igual a 24 horas': 'GRAVE',
    'Fallecido 24 horas': 'FALLECIDO'
}

In [13]:
df['lesividad'] = df['lesividad'].map(map_lesividad)

In [15]:
df['lesividad'].value_counts()

lesividad
NONE         30938
LEVE          7234
GRAVE         1494
FALLECIDO       24
Name: count, dtype: int64

creemos que nos va a dar mucho por culo esta columna

Mapeo Alcohol y drogas


In [17]:
df = df.fillna({
    'positivo_droga': 0
})

map_alcohol = {
    'N': 0,
    'S': 1
}

df['positiva_alcohol'] = df['positiva_alcohol'].map(map_alcohol)

In [18]:
df.head(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,Conductor,55,Hombre,2.0,GRAVE,444913.0,4481427.0,NaN,NaN
1,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Turismos y vehículos ligeros,Conductor,55,Mujer,14.0,NONE,444913.0,4481427.0,NaN,NaN
2,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Bicicletas y ciclos no motorizados,Conductor,25,Hombre,7.0,LEVE,440123.0,4475170.0,NaN,NaN
3,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,Conductor,35,Hombre,14.0,NONE,440123.0,4475170.0,NaN,NaN
4,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismos y vehículos ligeros,Conductor,25,Hombre,0.0,NONE,440137.0,4475721.0,NaN,NaN
5,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismos y vehículos ligeros,Conductor,55,Hombre,0.0,NONE,440137.0,4475721.0,NaN,NaN
6,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,Despejado,Vehículos de carga,Conductor,25,Hombre,14.0,NONE,440709.0,4480102.0,NaN,NaN
7,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,Despejado,Vehículos especializados,Conductor,50,Hombre,14.0,NONE,440709.0,4480102.0,NaN,NaN
8,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,Lluvia débil,Vehículos de carga,Conductor,60,Hombre,0.0,NONE,433983.0,4468473.0,NaN,NaN
9,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,Lluvia débil,Turismos y vehículos ligeros,Conductor,25,Hombre,0.0,NONE,433983.0,4468473.0,NaN,NaN


In [19]:
df['rango_edad'].value_counts()

rango_edad
-1     4316
 45    4107
 30    3919
 25    3885
 40    3845
 35    3796
 50    3698
 55    2869
 21    2627
 60    1990
 18    1268
 74     963
 65     956
 70     576
 15     374
 0      367
 10     349
 6      260
Name: count, dtype: int64

In [20]:
df.columns

Index(['num_expediente', 'fecha', 'hora', 'localizacion', 'numero',
       'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico',
       'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad',
       'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol',
       'positiva_droga'],
      dtype='object')

In [25]:
df['cod_lesividad'].value_counts()

cod_lesividad
0.0     17829
14.0    13109
7.0      4720
2.0      1494
1.0      1111
6.0       950
3.0       475
5.0       453
4.0        24
Name: count, dtype: int64

In [26]:
df.head()

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,Conductor,55,Hombre,2.0,GRAVE,444913.0,4481427.0,NaN,NaN
1,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Turismos y vehículos ligeros,Conductor,55,Mujer,14.0,NONE,444913.0,4481427.0,NaN,NaN
2,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Bicicletas y ciclos no motorizados,Conductor,25,Hombre,7.0,LEVE,440123.0,4475170.0,NaN,NaN
3,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,Conductor,35,Hombre,14.0,NONE,440123.0,4475170.0,NaN,NaN
4,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismos y vehículos ligeros,Conductor,25,Hombre,0.0,NONE,440137.0,4475721.0,NaN,NaN
